In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from abc import ABC, abstractmethod


In [ ]:
class MnistClassifierInterface(ABC):
    @abstractmethod
    def train(self, X_train, y_train, X_val=None, y_val=None):
        pass

    @abstractmethod
    def predict(self, X):
        pass

In [ ]:
class RandomForestMnistClassifier(MnistClassifierInterface):
    def __init__(self, n_estimators=100, random_state=52):

        self.n_estimators = n_estimators
        self.random_state = random_state
        self.model = RandomForestClassifier(n_estimators=self.n_estimators,
                                            random_state=self.random_state)

    def train(self, X_train, y_train, X_val=None, y_val=None):
        X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
        self.model.fit(X_train_reshaped, y_train)

        if X_val is not None and y_val is not None:
            X_val_reshaped = X_val.reshape(X_val.shape[0], -1)
            accuracy = self.model.score(X_val_reshaped, y_val)
            print(f"Validation accuracy (Random Forest): {accuracy:.4f}")

    def predict(self, X):
        X_reshaped = X.reshape(X.shape[0], -1)
        predictions = self.model.predict(X_reshaped)
        return predictions

In [ ]:
class NeuralNetworkMnistClassifier(MnistClassifierInterface):
    def __init__(self, input_shape=(784,), num_classes=10, epochs=5, batch_size=128):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None

        self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(128, activation='relu', input_shape=self.input_shape))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.num_classes, activation='softmax'))
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        self.model = model

    def train(self, X_train, y_train, X_val=None, y_val=None):
        X_train_reshaped = X_train.reshape(X_train.shape[0], -1).astype('float32')
        X_train_reshaped /= 255.0
        y_train_categorical = to_categorical(y_train, self.num_classes)

        if X_val is not None and y_val is not None:
            X_val_reshaped = X_val.reshape(X_val.shape[0], -1).astype('float32')
            X_val_reshaped /= 255.0
            y_val_categorical = to_categorical(y_val, self.num_classes)

            self.model.fit(
                X_train_reshaped, y_train_categorical,
                validation_data=(X_val_reshaped, y_val_categorical),
                epochs=self.epochs,
                batch_size=self.batch_size,
                verbose=1
            )
        else:
            self.model.fit(
                X_train_reshaped, y_train_categorical,
                epochs=self.epochs,
                batch_size=self.batch_size,
                verbose=1
            )

    def predict(self, X):
        X_reshaped = X.reshape(X.shape[0], -1).astype('float32')
        X_reshaped /= 255.0
        predictions_proba = self.model.predict(X_reshaped)
        predictions = np.argmax(predictions_proba, axis=1)
        return predictions

In [ ]:
class CNNMnistClassifier(MnistClassifierInterface):
    def __init__(self, input_shape=(28, 28, 1), num_classes=10, epochs=5, batch_size=128):
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = None

        self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=self.input_shape))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        self.model = model

    def train(self, X_train, y_train, X_val=None, y_val=None):
        if len(X_train.shape) < 4:
            X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
        X_train = X_train.astype('float32') / 255.0
        y_train_categorical = to_categorical(y_train, self.num_classes)

        if X_val is not None and y_val is not None:
            if len(X_val.shape) < 4:
                X_val = X_val.reshape(X_val.shape[0], 28, 28, 1)
            X_val = X_val.astype('float32') / 255.0
            y_val_categorical = to_categorical(y_val, self.num_classes)

            self.model.fit(
                X_train,
                y_train_categorical,
                validation_data=(X_val, y_val_categorical),
                epochs=self.epochs,
                batch_size=self.batch_size,
                verbose=1
            )
        else:
            self.model.fit(
                X_train,
                y_train_categorical,
                epochs=self.epochs,
                batch_size=self.batch_size,
                verbose=1
            )

    def predict(self, X):
        if len(X.shape) < 4:
            X = X.reshape(X.shape[0], 28, 28, 1)
        X = X.astype('float32') / 255.0
        predictions_proba = self.model.predict(X)
        predictions = np.argmax(predictions_proba, axis=1)
        return predictions


In [ ]:
class MnistClassifier:
    def __init__(self, algorithm='cnn', **kwargs):
        self.algorithm = algorithm
        self.model = None

        if algorithm == 'rf':
            self.model = RandomForestMnistClassifier(**kwargs)
        elif algorithm == 'nn':
            self.model = NeuralNetworkMnistClassifier(**kwargs)
        elif algorithm == 'cnn':
            self.model = CNNMnistClassifier(**kwargs)
        else:
            raise ValueError("Only 'rf', 'nn' or 'cnn'.")

    def train(self, X_train, y_train, X_val=None, y_val=None):
        self.model.train(X_train, y_train, X_val, y_val)

    def predict(self, X):
        return self.model.predict(X)


In [ ]:
def main():
    (X_train, y_train), (X_test, y_test) = mnist.load_data()
    X_val = X_train[-5000:]
    y_val = y_train[-5000:]
    X_train = X_train[:-5000]
    y_train = y_train[:-5000]

    rf_classifier = MnistClassifier(algorithm='rf', n_estimators=50, random_state=42)

    print("We train Random Forest...")
    rf_classifier.train(X_train, y_train, X_val, y_val)
    rf_predictions = rf_classifier.predict(X_test)
    rf_accuracy = np.mean(rf_predictions == y_test)
    print(f"Random Forest accuracy on test set: {rf_accuracy:.4f}\n")

    nn_classifier = MnistClassifier(algorithm='nn', epochs=3, batch_size=128)
    print("Training a fully connected neural network (NN)...")
    nn_classifier.train(X_train, y_train, X_val, y_val)
    nn_predictions = nn_classifier.predict(X_test)
    nn_accuracy = np.mean(nn_predictions == y_test)
    print(f"NN accuracy on test set: {nn_accuracy:.4f}\n")

    cnn_classifier = MnistClassifier(algorithm='cnn', epochs=3, batch_size=128)
    print("Training a convolutional neural network (CNN)...")
    cnn_classifier.train(X_train, y_train, X_val, y_val)
    cnn_predictions = cnn_classifier.predict(X_test)
    cnn_accuracy = np.mean(cnn_predictions == y_test)
    print(f"CNN accuracy on test set: {cnn_accuracy:.4f}")

if __name__ == "__main__":
    main()


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
We train Random Forest...
Validation accuracy (Random Forest): 0.9716
Random Forest accuracy on test set: 0.9675

Training a fully connected neural network (NN)...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8147 - loss: 0.6356 - val_accuracy: 0.9616 - val_loss: 0.1411
Epoch 2/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9558 - loss: 0.1511 - val_accuracy: 0.9728 - val_loss: 0.1002
Epoch 3/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.9706 - loss: 0.1015 - val_accuracy: 0.9728 - val_loss: 0.0950
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
NN accuracy on test set: 0.9699

Training a convolutional neural network (CNN)...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 44s 98ms/step - accuracy: 0.8521 - loss: 0.5096 - val_accuracy: 0.9780 - val_loss: 0.0752
Epoch 2/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 82s 98ms/step - accuracy: 0.9810 - loss: 0.0594 - val_accuracy: 0.9860 - val_loss: 0.0511
Epoch 3/3
430/430 ━━━━━━━━━━━━━━━━━━━━ 42s 97ms/step - accuracy: 0.9872 - loss: 0.0392 - val_accuracy: 0.9874 - val_loss: 0.0447
313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step
CNN accuracy on test set: 0.9860
